In [3]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import teamyearbyyearstats
from nba_api.stats.endpoints import leaguegamefinder

import matplotlib.pyplot
import pandas as pd
import time
import tensorflow

In [32]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
from datetime import datetime
#need to get list of all teams to avoid g league playoff games
nba_teams = teams.get_teams()
nba_team_ids = {team['id'] for team in nba_teams}
nba_team_abbreviations = {team['abbreviation'] for team in nba_teams}
# Set the season type to 'Playoffs'
season_type = 'Playoffs'

# Get the game logs for the playoffs
gamefinder = leaguegamefinder.LeagueGameFinder(season_type_nullable=season_type)
games = gamefinder.get_data_frames()[0]

# Filter the game logs to only include games from 2020 onwards
games['GAME_DATE'] = games['GAME_DATE'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
games_since_2000 = games[(games['GAME_DATE'].dt.year >= 2000)] #& (games['GAME_DATE'].dt.year <= 2020)]

games_since_2000['OPPONENT_ABBREVIATION'] = games_since_2000['MATCHUP'].apply(
    lambda x: x.split()[-1]
)

games_since_2000 = games_since_2000[
    (games_since_2000['TEAM_ABBREVIATION'].isin(nba_team_abbreviations)) & 
    (games_since_2000['OPPONENT_ABBREVIATION'].isin(nba_team_abbreviations))
]
games_since_2000

C:\Users\syedo\AppData\Local\Temp\ipykernel_7944\4101684608.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_since_2000['OPPONENT_ABBREVIATION'] = games_since_2000['MATCHUP'].apply(


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS,OPPONENT_ABBREVIATION
0,42023,1610612738,BOS,Boston Celtics,0042300405,2024-06-17,BOS vs. DAL,W,240,106,...,15,36,51,25,9,2,7,15,18.0,DAL
1,42023,1610612742,DAL,Dallas Mavericks,0042300405,2024-06-17,DAL @ BOS,L,240,88,...,7,28,35,18,4,4,13,20,-18.0,BOS
2,42023,1610612742,DAL,Dallas Mavericks,0042300404,2024-06-14,DAL vs. BOS,W,240,122,...,13,39,52,21,7,2,8,17,38.0,BOS
3,42023,1610612738,BOS,Boston Celtics,0042300404,2024-06-14,BOS @ DAL,L,239,84,...,4,27,31,18,2,5,13,19,-38.0,DAL
4,42023,1610612742,DAL,Dallas Mavericks,0042300403,2024-06-12,DAL vs. BOS,L,240,99,...,7,36,43,15,5,1,8,17,-7.0,BOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5543,41999,1610612749,MIL,Milwaukee Bucks,0049900008,2000-04-23,MIL @ IND,L,234,85,...,13,30,43,11,9,8,13,22,-3.0,IND
5545,41999,1610612759,SAS,San Antonio Spurs,0049900002,2000-04-22,SAS vs. PHX,L,236,70,...,9,41,50,15,7,9,18,19,-2.0,PHX
5547,41999,1610612748,MIA,Miami Heat,0049900001,2000-04-22,MIA vs. DET,W,236,95,...,7,28,35,21,9,11,12,29,10.0,DET
5549,41999,1610612756,PHX,Phoenix Suns,0049900002,2000-04-22,PHX @ SAS,W,235,72,...,10,37,47,20,10,10,14,20,2.0,SAS


In [33]:
# dictionary of all team ids to add to the data frame as away team ids
#team_ids = {team['id']: team['abbreviation'] for team in nba_teams}
team_ids = {team['abbreviation']: team['id'] for team in nba_teams}

games_since_2000['AWAY_TEAM_ID'] = games_since_2000['OPPONENT_ABBREVIATION'].map(team_ids)






games_since_2000 = games_since_2000.rename(columns={'TEAM_ABBREVIATION':'HOME_TEAM',
                                                    'OPPONENT_ABBREVIATION':'AWAY_TEAM',
                                                    'TEAM_ID': 'HOME_TEAM_ID'
                                                    })

rearranged_cols = ['SEASON_ID',
 'HOME_TEAM_ID',
 'AWAY_TEAM_ID',
 'HOME_TEAM',
 'AWAY_TEAM',
 'TEAM_NAME',
 'GAME_ID',
 'GAME_DATE',
 'MATCHUP',
 'WL',
 'MIN',
 'PTS',
 'FGM',
 'FGA',
 'FG_PCT',
 'FG3M',
 'FG3A',
 'FG3_PCT',
 'FTM',
 'FTA',
 'FT_PCT',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PLUS_MINUS',
 ]
games_since_2000 = games_since_2000[rearranged_cols]
games_since_2000

,SEASON_ID,HOME_TEAM_ID,AWAY_TEAM_ID,HOME_TEAM,AWAY_TEAM,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,42023,1610612738,1610612742,BOS,DAL,Boston Celtics,0042300405,2024-06-17,BOS vs. DAL,W,...,0.850,15,36,51,25,9,2,7,15,18.0
1,42023,1610612742,1610612738,DAL,BOS,Dallas Mavericks,0042300405,2024-06-17,DAL @ BOS,L,...,0.538,7,28,35,18,4,4,13,20,-18.0
2,42023,1610612742,1610612738,DAL,BOS,Dallas Mavericks,0042300404,2024-06-14,DAL vs. BOS,W,...,0.682,13,39,52,21,7,2,8,17,38.0
3,42023,1610612738,1610612742,BOS,DAL,Boston Celtics,0042300404,2024-06-14,BOS @ DAL,L,...,0.923,4,27,31,18,2,5,13,19,-38.0
4,42023,1610612742,1610612738,DAL,BOS,Dallas Mavericks,0042300403,2024-06-12,DAL vs. BOS,L,...,0.875,7,36,43,15,5,1,8,17,-7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5543,41999,1610612749,1610612754,MIL,IND,Milwaukee Bucks,0049900008,2000-04-23,MIL @ IND,L,...,0.808,13,30,43,11,9,8,13,22,-3.0
5545,41999,1610612759,1610612756,SAS,PHX,San Antonio Spurs,0049900002,2000-04-22,SAS vs. PHX,L,...,0.571,9,41,50,15,7,9,18,19,-2.0
5547,41999,1610612748,1610612765,MIA,DET,Miami Heat,0049900001,2000-04-22,MIA vs. DET,W,...,0.833,7,28,35,21,9,11,12,29,10.0
5549,41999,1610612756,1610612759,PHX,SAS,Phoenix Suns,0049900002,2000-04-22,PHX @ SAS,W,...,0.900,10,37,47,20,10,10,14,20,2.0


In [7]:

team_ids

{1610612737: 'ATL',
 1610612738: 'BOS',
 1610612739: 'CLE',
 1610612740: 'NOP',
 1610612741: 'CHI',
 1610612742: 'DAL',
 1610612743: 'DEN',
 1610612744: 'GSW',
 1610612745: 'HOU',
 1610612746: 'LAC',
 1610612747: 'LAL',
 1610612748: 'MIA',
 1610612749: 'MIL',
 1610612750: 'MIN',
 1610612751: 'BKN',
 1610612752: 'NYK',
 1610612753: 'ORL',
 1610612754: 'IND',
 1610612755: 'PHI',
 1610612756: 'PHX',
 1610612757: 'POR',
 1610612758: 'SAC',
 1610612759: 'SAS',
 1610612760: 'OKC',
 1610612761: 'TOR',
 1610612762: 'UTA',
 1610612763: 'MEM',
 1610612764: 'WAS',
 1610612765: 'DET',
 1610612766: 'CHA'}